To analyze and classify the quality of 3D meshes, I relied on several libraries. 
The trimesh library to process .obj 3D models and extract their geometric properties. 
For the classification itself, I used sklearn and tensorflow, which provided the tools to build and evaluate machine learning models.

In [1]:
import os
import numpy as np
import trimesh
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.ensemble import RandomForestClassifier


This section defines three functions for classifying the quality of 3D meshes based on specific geometric metrics:
- by Aspect Ratio
- by Height Ratio
- by Average Triangle Area

These functions assign a quality label (good, average, bad) to a 3D mesh based on specific metrics.

In [2]:
#Classify quality based on aspect ratio.
def classify_by_aspect_ratio(aspect_ratio):
    
    if aspect_ratio > 10:
        return "bad"
    elif 5 < aspect_ratio <= 10:
        return "average"
    else:
        return "good"


# Classify quality based on height ratio.
def classify_by_height_ratio(height_ratio):

    if height_ratio > 10:
        return "bad"
    elif 5 < height_ratio <= 10:
        return "average"
    else:
        return "good"
    


# Classify quality based on average triangle area
def classify_by_triangle_area(avg_area):

    if avg_area < 0.01:
        return "bad"
    elif 0.01 <= avg_area <= 0.1:
        return "average"
    else:
        return "good"
    



This function calculates the overall quality of a 3D mesh by analyzing geometric properties of its triangles.

It computes three metrics:
- aspect ratio, height ratio and average triangle area

and uses them to classify the mesh's quality.
The final quality is determined through majority voting based on these metrics.


In [3]:
# Compute quality based on multiple metrics
def compute_quality(mesh):

    aspect_ratios = []
    height_ratios = []  
    triangle_areas = []

    for face in mesh.faces:
        v0, v1, v2 = mesh.vertices[face]

        # Compute edge lengths
        edges = [
            np.linalg.norm(v1 - v0),
            np.linalg.norm(v2 - v1),
            np.linalg.norm(v0 - v2),
        ]
        longest_edge = max(edges)
        shortest_edge = min(edges)

        # Triangle area using Heron's formula
        s = sum(edges) / 2.0
        area = np.sqrt(s * (s - edges[0]) * (s - edges[1]) * (s - edges[2]))
        if area > 0:
            triangle_areas.append(area)

        # Compute aspect ratio
        if shortest_edge > 0:
            aspect_ratios.append(longest_edge / shortest_edge)

        # Compute all heights and find the longest and shortest
        heights = []
        for edge in edges:
            if edge > 0:  # Avoid division by zero
                height = 2 * area / edge
                heights.append(height)

        if len(heights) == 3:  # Ensure all heights are valid
            longest_height = max(heights)
            shortest_height = min(heights)
            if shortest_height > 0:  # Avoid division by zero 
                height_ratios.append(longest_height / shortest_height)

    # Compute average metrics
    avg_aspect_ratio = np.mean(aspect_ratios) if aspect_ratios else float('nan')
    avg_height_ratio = np.mean(height_ratios) if height_ratios else float('nan')
    avg_triangle_area = np.mean(triangle_areas) if triangle_areas else float('nan')

    # Vote for quality
    votes = [
        classify_by_aspect_ratio(avg_aspect_ratio),
        classify_by_height_ratio(avg_height_ratio),
        classify_by_triangle_area(avg_triangle_area),
    ]

    # Majority voting
    vote_counts = {vote: votes.count(vote) for vote in set(votes)}
    most_common_vote = max(vote_counts, key=vote_counts.get)

    # Section that decides on the quality in the event of a tie
    if list(vote_counts.values()).count(vote_counts[most_common_vote]) > 1:
        return "average"  # Default to average in case of a tie
    return most_common_vote


This function loads a 3D mesh from a file using the trimesh library. It ensures that the loaded file is a valid 3D mesh and raises appropriate errors if the file cannot be loaded or is invalid.

In [4]:
# Loading a mesh from a file using trimesh
def load_mesh(file_path):

    try:
        mesh = trimesh.load(file_path)
        if not isinstance(mesh, trimesh.Trimesh):
            raise ValueError("Not a valid Trimesh object.")
        if len(mesh.vertices) == 0 or len(mesh.faces) == 0:
            raise ValueError("Mesh has no vertices or faces.")
        return mesh
    except Exception as e:
        raise ValueError(f"Error loading mesh from {file_path}: {e}")


This function converts a 3D mesh into fixed-size matrices. The function ensures the data is normalized, padded or truncated to meet the specified size limits for vertices and faces.

In [5]:
# Convertion of mesh into fixed-size matrices.
def extract_mesh_to_matrix(mesh, max_vertices=500000, max_faces=1000000):
   
    vertices = mesh.vertices
    faces = mesh.faces

    # Normalize vertices (center and scale)
    vertices = vertices - np.mean(vertices, axis=0)
    vertices = vertices / np.max(np.linalg.norm(vertices, axis=1))

    # Pad or truncate vertices
    if len(vertices) > max_vertices:
        vertices = vertices[:max_vertices]
    else:
        vertices = np.pad(vertices, ((0, max_vertices - len(vertices)), (0, 0)), 'constant')

    # Pad or truncate faces
    if len(faces) > max_faces:
        faces = faces[:max_faces]
    else:
        faces = np.pad(faces, ((0, max_faces - len(faces)), (0, 0)), 'constant')

    # Flatten vertices and faces into a single array
    matrix = np.hstack([vertices.flatten(), faces.flatten()])
    return matrix


This function processes all 3D meshes in a specified directory, computes quality metrics for each mesh, and organizes the data for further use. It uses previously defined functions (load_mesh, extract_mesh_to_matrix, and compute_quality) to handle each mesh.

In [6]:
# Process all meshes in the directory and compute quality metrics.
def process_meshes(directory_path):
  
    mesh_data = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if file_name.lower().endswith('.obj'):
            try:
                mesh = load_mesh(file_path)
                matrix = extract_mesh_to_matrix(mesh)

                # Compute quality based on votes
                quality_group = compute_quality(mesh)

                mesh_data.append({
                    "file_name": file_name,
                    "matrix": matrix,
                    "quality_group": quality_group
                })
                print(f"Processed: {file_name} -> Quality: {quality_group}")
            except Exception as e:
                print(f"Failed to process {file_name}: {e}")
    return mesh_data

This function trains and evaluates a Random Forest Classifier on the processed mesh data. It predicts the quality group of each mesh (e.g., good, average, bad) based on the feature matrix extracted from the meshes.

In [7]:
def train_random_forest(mesh_data):
    # Prepare dataset
    X = np.array([data["matrix"] for data in mesh_data])
    y = np.array([data["quality_group"] for data in mesh_data])

    # Encode labels to numeric values
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Splitting data into training/testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # Train Random Forest Classifier 
    rf_clf = RandomForestClassifier(random_state=42)
    rf_clf.fit(X_train, y_train)

    # Test Random Forest Classifier
    rf_pred = rf_clf.predict(X_test)
    print("\nRandom Forest Classification Report:")
    print(classification_report(y_test, rf_pred, target_names=label_encoder.classes_))
    rf_accuracy = accuracy_score(y_test, rf_pred)
    print(f"Random Forest Test Accuracy: {rf_accuracy:.2f}")

    return rf_clf, label_encoder


This function trains a neural network to classify the quality of 3D meshes. It uses PCA to reduce the dimensionality of the input data. The network uses progressively smaller dense layers with dropout to prevent overfitting. 

In [33]:
def train_neural_network(mesh_data, n_components=138, batch_size=32, epochs=20):
    # Prepare dataset
    X = np.array([data["matrix"] for data in mesh_data], dtype=np.float32)
    y = np.array([data["quality_group"] for data in mesh_data])

    # Dimensionality reduction using PCA
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)

    # Encode labels to numeric values
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Splitting data into training/testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_encoded, test_size=0.2, random_state=42)

    # Neural Network Architecture
    nn_model = Sequential([
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    # Compile the model
    nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Training and tracking of the model
    history = nn_model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Final test evaluation
    nn_loss, nn_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"\nFinal Neural Network Test Accuracy: {nn_accuracy:.2f}")

    return nn_model, label_encoder, pca, history

The following section contains the code that processes the meshes from the given directory.

In [9]:
directory_path = "3d meshe"  # Path to directory with meshes

# Process meshes
mesh_data = process_meshes(directory_path)

Processed: 5. American Gothic by Grant Wood.obj -> Quality: average
Processed: AK47.obj -> Quality: bad
Processed: Ammo_Box.obj -> Quality: bad
Processed: AT_MINE.obj -> Quality: bad
Processed: BaseballBat.obj -> Quality: bad
Processed: BathroomSink.obj -> Quality: average
Processed: Bear.OBJ -> Quality: good
Processed: BearTrap.obj -> Quality: bad
Processed: Boxer.obj -> Quality: average
Processed: C4.obj -> Quality: bad
Processed: Ceiling_Lamp.obj -> Quality: average
Processed: Christ the Redeemer.obj -> Quality: good
Processed: Claymore.obj -> Quality: bad
Processed: Closet.obj -> Quality: average
Processed: Coffee_Table.obj -> Quality: bad
Processed: Combat_Knife.obj -> Quality: average
Processed: Console.obj -> Quality: bad
Processed: Container.obj -> Quality: bad
Processed: Crate.obj -> Quality: average
Processed: Cupboard.obj -> Quality: average
Processed: Deer.obj -> Quality: good
Processed: Door01.obj -> Quality: bad
Processed: Drone.obj -> Quality: average
Processed: Dynamite

C:\Users\marti\AppData\Local\Temp\ipykernel_11912\4174747390.py:22: RuntimeWarning: invalid value encountered in sqrt
  area = np.sqrt(s * (s - edges[0]) * (s - edges[1]) * (s - edges[2]))


Processed: Grease_Gun.obj -> Quality: average
Processed: hand1.OBJ -> Quality: good
Processed: happy_budha-original.obj -> Quality: average
Processed: IKEA_BESTA_1.obj -> Quality: bad
Processed: IKEA_BILLY_1.obj -> Quality: bad
Processed: IKEA_BILLY_2.obj -> Quality: bad
Processed: IKEA_BILLY_3.obj -> Quality: bad
Processed: IKEA_BILLY_4.obj -> Quality: bad
Processed: IKEA_BILLY_5.obj -> Quality: bad
Processed: IKEA_BJORKUDDEN_1.obj -> Quality: bad
Processed: IKEA_BRIMNES_1.obj -> Quality: bad
Processed: IKEA_BRIMNES_2.obj -> Quality: bad
Processed: IKEA_EKTORP_2.obj -> Quality: bad
Processed: IKEA_EKTORP_3.obj -> Quality: bad
Processed: IKEA_EXPEDIT_2.obj -> Quality: average
Processed: IKEA_EXPEDIT_3.obj -> Quality: bad
Processed: IKEA_KAUSTBY.obj -> Quality: bad
Processed: IKEA_LACK_1.obj -> Quality: average
Processed: IKEA_LACK_2.obj -> Quality: average
Processed: IKEA_LAIVA.obj -> Quality: bad
Processed: IKEA_MANSTAD.obj -> Quality: bad
Processed: IKEA_NORDEN_1.obj -> Quality: bad


A code for starting training a Random Forest model on mesh data.

In [24]:
if len(mesh_data) > 0:
    rf_clf, label_encoder_1 = train_random_forest(mesh_data)
else:
    print("No valid meshes to process.")


Random Forest Classification Report:
              precision    recall  f1-score   support

     average       0.67      0.67      0.67         6
         bad       0.70      0.88      0.78         8
        good       1.00      0.86      0.92        14

    accuracy                           0.82        28
   macro avg       0.79      0.80      0.79        28
weighted avg       0.84      0.82      0.83        28

Random Forest Test Accuracy: 0.82


A code for starting training a Neural Network model on mesh data.

In [39]:
if len(mesh_data) > 0:
    nn_model, label_encoder_2, pca, history= train_neural_network(mesh_data)
else:
    print("No valid meshes to process.")

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 0.3424 - loss: 3269662.0000 - val_accuracy: 0.4286 - val_loss: 1575136.3750
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4092 - loss: 2401079.5000 - val_accuracy: 0.3929 - val_loss: 616045.5000
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3613 - loss: 3709973.7500 - val_accuracy: 0.5000 - val_loss: 107906.3828
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4320 - loss: 2106746.2500 - val_accuracy: 0.3929 - val_loss: 203259.9844
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4774 - loss: 2907929.5000 - val_accuracy: 0.3929 - val_loss: 293655.7188
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4372 - loss: 721755.1250 - val_accuracy: 0.4286 - val_loss: 157220.5938
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4549 - loss: 754321.5625 - val_accuracy: 0.4286 - val_loss: 180560.2500
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step 

A small example of how the quality of 3d meshes can be predicted with trained models

In [40]:
directory = "meshe na ukazku"  # Directory containing meshes
mesh_data_2 = process_meshes(directory)

print()
# Preparing the feature matrix, object names and real quality
X = np.array([data["matrix"] for data in mesh_data_2])
names = [data["file_name"] for data in mesh_data_2]
real_qualities = [data["quality_group"] for data in mesh_data_2]  


# Prediction using the trained Random Forest model
predicted_indices_rf = rf_clf.predict(X)
predicted_labels_rf = label_encoder_1.inverse_transform(predicted_indices_rf)

# Prediction using the trained Neural Network model
predicted_probabilities_nn = nn_model.predict(pca.transform(X))
predicted_indices_nn = np.argmax(predicted_probabilities_nn, axis=1)
predicted_labels_nn = label_encoder_2.inverse_transform(predicted_indices_nn)

print()
# Print predictions 
for i in range(len(names)):
    print(f"Object: {names[i]}:")
    print(f"  Real Quality: {real_qualities[i]}")
    print(f"  Predicted Quality (Random Forest): {predicted_labels_rf[i]}")
    print(f"  Predicted Quality (Neural Network): {predicted_labels_nn[i]}\n")


Processed: Doctor.obj -> Quality: good
Processed: Frog.obj -> Quality: good
Processed: Saw.obj -> Quality: bad
Processed: Squirrel.OBJ -> Quality: good

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step

Object: Doctor.obj:
  Real Quality: good
  Predicted Quality (Random Forest): good
  Predicted Quality (Neural Network): good

Object: Frog.obj:
  Real Quality: good
  Predicted Quality (Random Forest): average
  Predicted Quality (Neural Network): bad

Object: Saw.obj:
  Real Quality: bad
  Predicted Quality (Random Forest): bad
  Predicted Quality (Neural Network): bad

Object: Squirrel.OBJ:
  Real Quality: good
  Predicted Quality (Random Forest): good
  Predicted Quality (Neural Network): good

